# 네이버 뉴스 추천 시스템 구현해보기

- (DC1) 실시간으로 사용자의 선호도 예측
- (DC2) 자동화 방식의 뉴스 품질 측정
- (DC3) 시의 적절한 주요 이슈 감지
- (DC4) 확장성 있는 시스템 구조

(S1) 후보 뉴스 기사 생성 단계 
- CF-based Generation (DC1)
- QE-based Generation (DC2)
- SI(user)-based Generation (DC3)
- SI(press)-based Generation (DC3)

## 변수 설명

사용자 ui가 최근 x 일 동안 소비한 각 뉴스 기사 vj, 소비하지 않은 뉴스 기사들 vk  
P(vj)와 P(vk) 각각은 x일 동안 전체 사용자의 뉴스 소비 로그에서 뉴스 기사 vj, vk 각각의 소비 비율을 나타내고, P(vj, vk)은 vj, vk를 함께 소비한 비율을 나타냅니다.

## 구현 과정

1. behaviors.tsv 파일을 불러옵니다.
2. history 순회를 돌며 P(vj), P(vk), P(vj, vk)에 대한 sparse matrix를 생성합니다.
3. NPMI(vj, vk)를 생성합니다.

### 조건
matrix 크기를 줄이기 위해 항상 vj < vk로 취급합니다. vj == vk인 경우는 없습니다.

In [1]:
# jupyter notebook에서 import 해서 쓰는 모듈의 코드가 변경될 시, 변동 사항을 자동으로 반영해주는 기능 켜기
%load_ext autoreload
%autoreload 2

In [1]:
import os
from os import path
import sys

DATASET_SIZE = "small"
PROJECT_DIR = path.abspath(path.join(os.getcwd(), "..", ".."))
DATASET_DIR = path.join(PROJECT_DIR, "data", "MIND", DATASET_SIZE)

sys.path.append(PROJECT_DIR)

import torch

from npmi_newsrec import NPMINewsrec

c:\Users\nclud\anaconda3\envs\newsrec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
newsrec = NPMINewsrec(DATASET_SIZE)

NPMI 계산 전, history와 impressions에 대한 전처리를 진행합니다.
1. news2int를 생성합니다.
2. Dataframe의 history를 정리합니다.
3. Dataframe의 impressions를 정리합니다.
전처리를 완료했습니다.
각 뉴스와 뉴스 쌍의 소비 비율을 계산하기 위해, history에서의 등장 횟수를 측정합니다.(중복 제외)


100%|██████████| 156965/156965 [02:01<00:00, 1289.82it/s]


측정한 횟수를 바탕으로 모든 뉴스 쌍의 NPMI 점수를 계산합니다.
모든 뉴스 pair에 대한 npmi 점수를 저장한 dict를 생성합니다.


100%|██████████| 11107456/11107456 [01:58<00:00, 93628.86it/s]


생성을 완료했습니다.
batch를 생성합니다.
train 데이터셋으로 계산한 뉴스 쌍의 NPMI 점수와 모든 사용자의 history를 기반으로, impression의 각 뉴스에 대한 최대 NPMI 점수를 찾습니다.


100%|██████████| 156965/156965 [01:52<00:00, 1393.27it/s]


batch생성을 완료했습니다.


In [8]:
test_result = newsrec.run_eval(0.9)
print(test_result)

100%|██████████| 156965/156965 [00:02<00:00, 74277.79it/s]

Used 184/156965 samples.
{'val_auc': tensor(0.5665), 'val_mrr': tensor(0.6393), 'val_ndcg@10': tensor(0.7082), 'val_ndcg@5': tensor(0.7082)}


In [4]:
test_result_rand = newsrec.run_eval_random()
print(test_result_rand)

100%|██████████| 156965/156965 [07:29<00:00, 349.48it/s]

{'val_auc': tensor(0.4997), 'val_mrr': tensor(0.2295), 'val_ndcg@10': tensor(0.2928), 'val_ndcg@5': tensor(0.2928)}


In [12]:
newsrec.dataset.npmi_dict[(4, 36)]

0.02623668871819973

## imp logs
1. negative sampling
2. torchmetrics

In [13]:
newsrec.check_npmi(40)

(2, 26): 0.2648686170578003
(2, 36): 0.040272701531648636
(4, 26): 0.02107863686978817
(4, 36): 0.02623668871819973
(15, 36): 0.15379023551940918
(26, 30): 0.15547339618206024
(26, 36): 0.00703651225194335
(26, 39): 0.48407480120658875
(29, 36): 0.10140280425548553
(30, 36): 0.07632788270711899


In [10]:
batch = next(iter(newsrec.dataloader))

In [11]:
batch["pairs_npmi"].shape
# batch_size, impr_num, history_num, pair_size 

torch.Size([1, 87, 6])

In [ ]:
batch

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def pad_fixed_length(seqs: list[torch.Tensor], max_len: int, padding_value: int = 0):
    # 먼저 자르기
    clipped_seqs = [s[:max_len] for s in seqs]

    # 패딩 적용
    padded = pad_sequence(clipped_seqs, batch_first=True, padding_value=padding_value)

    # 필요하면 오른쪽 끝까지 패딩 추가
    if padded.shape[1] < max_len:
        padded = torch.nn.functional.pad(
            padded, (0, max_len - padded.shape[1]), value=padding_value
        )

    return padded

history_idxs = [torch.tensor(history_idxs, dtype=torch.int32) for history_idxs in newsrec.dataset.train_behaviors_df["history_idxs"]]
impr_idxs = [torch.tensor(impr_idxs, dtype=torch.int32) for impr_idxs in newsrec.dataset.train_behaviors_df["impr_idxs"]]
history_idxs = pad_fixed_length(history_idxs, max_len=200)
impr_idxs = pad_fixed_length(impr_idxs, max_len=200)

In [ ]:
newsrec.dataset.batch

In [21]:
print(newsrec.dataset.batch[0]["scores"].tolist())
print(newsrec.dataset.batch[0]["labels"].shape)

[-1.0, -1.0, -1.0, 0.1899547278881073, 0.17113995552062988, -1.0, -1.0, -1.0, -1.0, 0.2296501249074936, -1.0, -1.0, 0.2539237439632416, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
torch.Size([19])


In [15]:
import csv

with open("batch_MINDlarge.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["batch_idx", "score", "label"])
    
    for batch_idx, entry in enumerate(newsrec.dataset.batch):
        scores = entry["scores"]
        labels = entry["labels"]

        scores = entry["scores"].tolist()
        labels = entry["labels"].tolist()
        writer.writerow([batch_idx, scores, labels])

        if batch_idx > 100:
            break


In [10]:
import pickle

# 저장
with open("batch_MINDlarge.pkl", "wb") as f:
    pickle.dump(newsrec.dataset.batch, f)

In [ ]:
# 불러오기
with open("npmi_dict.pkl", "rb") as f:
    npmi_dict = pickle.load(f)